In [3]:
from bs4 import BeautifulSoup
import requests
import pymongo

In [4]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

In [8]:
cine21_url = "http://www.cine21.com/rank/person/content"
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2019-07'
post_data['gender'] = 'all'
post_data['page'] = 1

In [9]:
res = requests.post(cine21_url, data=post_data)

In [10]:
soup = BeautifulSoup(res.content, 'html.parser')

In [18]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(actor.get_text().split('(')[0])

이병헌
하정우
마동석
전혜진
이성민
김아중
박정민


In [23]:
import re # 정규표현식
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

이병헌
하정우
마동석
전혜진
이성민
김아중
박정민


In [46]:
actors = soup.select('li.people_li div.name')
actors_info_list = list()
for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
    res_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(res_actor.content, 'html.parser')
    actor_details = soup_actor.select('ul.default_info li')
    
    actor_info_dict = dict()
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value
    actors_info_list.append(actor_info_dict)
    
actors_info_list

[{'다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'다른 이름': 'Ma Dongseok',
  '직업': '배우',
  '생년월일': '1971-03-01',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n',
  '신장/체중': '178cm, 100kg'},
 {'다른 이름': '전이다',
  '직업': '배우',
  '생년월일': '1976-08-10',
  '성별': '여',
  '신장/체중': '171cm, 49kg',
  '학교': '상명대학교 영화과',
  '취미': '영화감상, 노래 부르기',
  '특기': '수영',
  '소속사': '호두엔터테인먼트'},
 {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'},
 {'직업': '배우',
  '생년월일': '1982-10-16',
  '성별': '여',
  '홈페이지': '\nhttps://twitter.com/ajoongkim\nhttps://instagram.com/ashia_kim/\n

In [59]:
actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li> strong')
movies = soup.select('ul.mov_list')
for index, actor in enumerate(actors):
    print("배우이름 :", re.sub('\(\w*\)', '', actor.text))
    print("흥행지수 :", int(hits[index].text.replace(',', '')))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    print("출연영화 :", movie_title_list)

배우이름 : 이병헌
흥행지수 : 46630
출연영화 : ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다']
배우이름 : 하정우
흥행지수 : 31716
출연영화 : ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨']
배우이름 : 마동석
흥행지수 : 19381
출연영화 : ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']
배우이름 : 전혜진
흥행지수 : 18318
출연영화 : ['백두산', '비스트', '뺑반', '불한당: 나쁜 놈들의 세상', '택시운전사']
배우이름 : 이성민
흥행지수 : 15503
출연영화 : ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']
배우이름 : 김아중
흥행지수 : 15497
출연영화 : ['나쁜 녀석들: 더 무비', '더 킹']
배우이름 : 박정민
흥행지수 : 14804
출연영화 : ['시동', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '다만 악에서 구하소서', '동주']


In [60]:
actors_info_list = list()

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li> strong')
movies = soup.select('ul.mov_list')
    
for index, actor in enumerate(actors):
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_hits = int(hits[index].text.replace(',', ''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    
    actor_info_dict = dict()
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_title_list
    
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
    res_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(res_actor.content, 'html.parser')
    actor_details = soup_actor.select('ul.default_info li')
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value
    actors_info_list.append(actor_info_dict)
    
actors_info_list

[{'배우이름': '이병헌',
  '흥행지수': 46630,
  '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'],
  '다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'배우이름': '하정우',
  '흥행지수': 31716,
  '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'],
  '다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'배우이름': '마동석',
  '흥행지수': 19381,
  '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'],
  '다른 이름': 'Ma Dongseok',
  '직업': '배우',
  '생년월일': '1971-03-01',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n',
  '신장/체중': '178cm, 100kg'},
 {'배우이름': '전혜진',
  '흥행지수': 18318,
  '출연영화'

In [64]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actors_info_list = list()

for i in range(1, 21):
    cine21_url = "http://www.cine21.com/rank/person/content"
    post_data = dict()
    post_data['section'] = 'actor'
    post_data['period_start'] = '2019-07'
    post_data['gender'] = 'all'
    post_data['page'] = i
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')
    

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li> strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        actor_ranking = rankings[index].text
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['순위'] = actor_ranking

        actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
        res_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(res_actor.content, 'html.parser')
        actor_details = soup_actor.select('ul.default_info li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)

actor_collection.insert_many(actors_info_list)

[{'배우이름': '이병헌',
  '흥행지수': 46630,
  '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'],
  '순위': '1',
  '다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'배우이름': '하정우',
  '흥행지수': 31716,
  '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'],
  '순위': '2',
  '다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'배우이름': '마동석',
  '흥행지수': 19381,
  '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'],
  '순위': '3',
  '다른 이름': 'Ma Dongseok',
  '직업': '배우',
  '생년월일': '1971-03-01',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n',
  '신장/체중': '178cm, 100kg'},
 {'

In [65]:
actor_collection.insert_many(actors_info_list)

In [43]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection
actor = actor_collection

In [90]:
docs = actor.find({'흥행지수':{'$gte' : 10000}, '$or' : [{'출연영화':'엑시트'}, {'출연영화':'시동'}]}, {'배우이름':1, '출연영화':1, '흥행지수':1, '_id':0}).sort('흥행지수', pymongo.DESCENDING)
for doc in docs:
    print(doc)

{'배우이름': '박정민', '흥행지수': 14804, '출연영화': ['시동', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '다만 악에서 구하소서', '동주']}
{'배우이름': '강기영', '흥행지수': 14240, '출연영화': ['엑시트', '가장 보통의 연애', '퍼즐']}
{'배우이름': '조정석', '흥행지수': 13443, '출연영화': ['엑시트', '형', '시간이탈자', '특종: 량첸살인기', '뺑반', '나의 사랑 나의 신부']}
{'배우이름': '임윤아', '흥행지수': 12577, '출연영화': ['엑시트']}
{'배우이름': '고두심', '흥행지수': 11714, '출연영화': ['엑시트']}
{'배우이름': '박인환', '흥행지수': 10871, '출연영화': ['엑시트', '봄날은 간다', '비밥바룰라', '기묘한 가족', '수상한 그녀']}
{'배우이름': '정해인', '흥행지수': 10645, '출연영화': ['시동', '유열의 음악앨범', '장수상회']}
{'배우이름': '김지영', '흥행지수': 10035, '출연영화': ['엑시트', '프랑스여자']}


In [97]:
docs = actor.find({'$nor':[{'흥행지수':{'$gte':10000}},{'흥행지수':{'$lte':8000}}]},{'배우이름':1, '흥행지수':1, '_id':0})
for doc in docs:
    print(doc)

{'배우이름': '이봉련', '흥행지수': 9576}
{'배우이름': '조우진', '흥행지수': 8891}
{'배우이름': '공유', '흥행지수': 8792}
{'배우이름': '김종구', '흥행지수': 8308}
{'배우이름': '박지환', '흥행지수': 8129}


In [99]:
docs = actor.find({'흥행지수':{'$in' : [9576, 8891, 8792]}},{'배우이름':1, '흥행지수':1, '_id':0})
for doc in docs:
    print(doc)

{'배우이름': '이봉련', '흥행지수': 9576}
{'배우이름': '조우진', '흥행지수': 8891}
{'배우이름': '공유', '흥행지수': 8792}


In [102]:
docs = actor.find({'$nor':[{'흥행지수':{'$in':[9182,8439]}},{'흥행지수':{'$gt':10000}}]},{'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print(doc)

{'배우이름': '이봉련', '흥행지수': 9576}
{'배우이름': '조우진', '흥행지수': 8891}
{'배우이름': '공유', '흥행지수': 8792}


In [107]:
docs = actor.find({'출연영화':{'$in' : ['사바하','극한직업']}},{'배우이름':1, '흥행지수':1,'출연영화':1, '_id':0})
for doc in docs:
    print(doc) # 사바하 또는 극한직업에 출연한 배우

{'배우이름': '이하늬', '흥행지수': 6571, '출연영화': ['블랙머니', '극한직업', '조작된 도시']}
{'배우이름': '김종수', '흥행지수': 4764, '출연영화': ['시동', '나를 찾아줘', '극한직업', '밀양', '돈', '증인']}
{'배우이름': '이재인', '흥행지수': 4464, '출연영화': ['봉오동 전투', '아워 바디', '사바하', '어른도감', '아이 캔 스피크']}
{'배우이름': '진선규', '흥행지수': 2116, '출연영화': ['롱 리브 더 킹: 목포 영웅', '퍼펙트맨', '암전', '극한직업', '로망', '암수살인']}


In [109]:
docs = actor.find({'출연영화':{'$all' : ['블랙머니','극한직업']}},{'배우이름':1, '흥행지수':1,'출연영화':1, '_id':0})
for doc in docs:
    print(doc) # 블랙머니와 극한직업에 동시에 출연한 배우

{'배우이름': '이하늬', '흥행지수': 6571, '출연영화': ['블랙머니', '극한직업', '조작된 도시']}


In [116]:
docs = actor.find({'출연영화.0':'시동'})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c5954c'), '배우이름': '박정민', '흥행지수': 14804, '출연영화': ['시동', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '다만 악에서 구하소서', '동주'], '순위': '7', '직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}
{'_id': ObjectId('5f17c4855d03ae9996c5955a'), '배우이름': '정해인', '흥행지수': 10645, '출연영화': ['시동', '유열의 음악앨범', '장수상회'], '순위': '21', '직업': '배우', '생년월일': '1988-04-01', '성별': '남', '홈페이지': '\nhttps://twitter.com/actorhaein\nhttps://instagram.com/holyhaein/\n'}
{'_id': ObjectId('5f17c4855d03ae9996c5956c'), '배우이름': '염정아', '흥행지수': 6822, '출연영화': ['시동', '범죄의 재구성', '뺑반', '강철비2: 정상회담', '미성년', '오래된 정원'], '순위': '39', '직업': '배우', '성별': '여', '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/yum-jung-ah/\n', '소속사': '아티스트컴퍼니'}
{'_id': ObjectId('5f17c4855d03ae9996c59574'), '배우이름': '최성은', '흥행지수': 5682, '출연영화': ['시동'], '순위': '47', '직업': '배우', '생년월일': '1996-06-17', '성별': '여'}
{'_id': ObjectId('5f17c4855d03ae9996c59577'), '배우이름': '윤경호', '흥행지수': 5657, '

In [122]:
docs = actor.find({'출연영화':{'$size':1}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c59551'), '배우이름': '임윤아', '흥행지수': 12577, '출연영화': ['엑시트'], '순위': '12', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}
{'_id': ObjectId('5f17c4855d03ae9996c59553'), '배우이름': '고두심', '흥행지수': 11714, '출연영화': ['엑시트'], '순위': '14', '직업': '배우', '생년월일': '1951-05-22', '성별': '여', '홈페이지': '\nhttp://www.kodoosim.com/\n', '신장/체중': '160cm', '학교': '제주여자고등학교'}
{'_id': ObjectId('5f17c4855d03ae9996c59554'), '배우이름': '장기용', '흥행지수': 11623, '출연영화': ['나쁜 녀석들: 더 무비'], '순위': '15', '직업': '배우', '생년월일': '1992-08-07', '성별': '남', '홈페이지': '\nhttps://instagram.com/juanxkui\n'}
{'_id': ObjectId('5f17c4855d03ae9996c59563'), '배우이름': '김상중', '흥행지수': 7749, '출연영화': ['나쁜 녀석들: 더 무비'], '순위': '30', '직업': '배우', '생년월일': '1965-08-06', '성별': '남', '신장/체중': '175cm, 65kg', '학교': '동국대학교 연극영화학', '소속사': '(주)주머니필름'}
{'_id': ObjectId('5f17c4855d03ae9996c59565'), '배우이름': '김미경', '흥행지수': 7536,

In [142]:
docs = actor.find({'직업':'가수'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c59550'), '배우이름': '배수지', '흥행지수': 12886, '출연영화': ['백두산', '건축학개론'], '순위': '11', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': '\nhttps://twitter.com/missa_suzy\nhttps://www.instagram.com/skuukzky/\n', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트', '다른이름': '배수지;미쓰에이;'}
{'_id': ObjectId('5f17c4855d03ae9996c59551'), '배우이름': '임윤아', '흥행지수': 12577, '출연영화': ['엑시트'], '순위': '12', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}
{'_id': ObjectId('5f17c4855d03ae9996c595c3'), '배우이름': '최민호', '흥행지수': 2046, '출연영화': ['장사리 : 잊혀진 영웅들', '인랑'], '순위': '126', '직업': '가수', '생년월일': '1991-12-09', '성별': '남', '신장/체중': '181cm', '학교': '건국대학교 예술학', '취미': '축구, 농구', '특기': '연기, 영어, 중국어', '소속사': 'SM엔터테인먼트', '다른이름': '珉豪; 샤이니; SHINee'}


In [143]:
docs = actor.find({'출연영화':'국가부도의 날'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c59552'), '배우이름': '유아인', '흥행지수': 12503, '출연영화': ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말'], '순위': '13', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/hongsick/\nhttps://twitter.com/seeksik\n', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영', '다른이름': '엄홍식'}
{'_id': ObjectId('5f17c4855d03ae9996c5955f'), '배우이름': '조우진', '흥행지수': 8891, '출연영화': ['봉오동 전투', '내부자들', '돈', '국가부도의 날', '어쩌다, 결혼', '창궐'], '순위': '26', '직업': '배우', '생년월일': '1979-01-16', '성별': '남', '다른이름': '조신제'}
{'_id': ObjectId('5f17c4855d03ae9996c5956b'), '배우이름': '허준호', '흥행지수': 6866, '출연영화': ['천문: 하늘에 묻는다', '퍼펙트맨', '결백', '국가부도의 날', '인랑'], '순위': '38', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}
{'_id': ObjectId('5f17c4855d03ae9996c595cf'), '배우이름': '송영창', '흥행지수': 1748, '출연영화': ['정직한 후보', '변호인', '베테랑', '말모이', '국

### elemMatch (적어도 한개 이상의 리스트 요소가 복수개의 조건을 동시에 만족하는 경우)

In [123]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
sample = actor_db.sample

In [134]:
sample.insert_many([
    {'results':[82, 85, 88]},
    {'results':[75, 88, 91]}
])

In [135]:
docs = sample.find({'results':{"$gte":90, '$lt':85}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17ec9d5d03ae9996c595d9'), 'results': [75, 88, 91]}


In [138]:
docs = sample.find({'results':{'$elemMatch':{'$gte':80, '$lt':85}}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17ec9d5d03ae9996c595d8'), 'results': [82, 85, 88]}


### 텍스트 검색과 인덱스

In [5]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_collection'},
 '출연영화_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine21.actor_collection',
  'weights': SON([('출연영화', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [157]:
actor.create_index([('출연영화', 'text')])

'출연영화_text'

In [15]:
docs = actor.find({'$text' : {'$search' : '신과'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('5f17c4855d03ae9996c595ce'), '배우이름': '정지훈', '흥행지수': 1758, '출연영화': ['사자', '덕구', '신과 함께-인과 연'], '순위': '137', '직업': '배우', '생년월일': '2007-05-31', '성별': '남'}
{'_id': ObjectId('5f17c4855d03ae9996c59548'), '배우이름': '마동석', '흥행지수': 19381, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'], '순위': '3', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg', '다른이름': 'Ma Dongseok'}


In [19]:
actor.drop_indexes()

In [17]:
actor.create_index([('직업', 'text')]) # text형식의 인덱스는 하나만 존재

'직업_text'

In [18]:
docs = actor.find({'$text' : {'$search' : '가수'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c595c3'), '배우이름': '최민호', '흥행지수': 2046, '출연영화': ['장사리 : 잊혀진 영웅들', '인랑'], '순위': '126', '직업': '가수', '생년월일': '1991-12-09', '성별': '남', '신장/체중': '181cm', '학교': '건국대학교 예술학', '취미': '축구, 농구', '특기': '연기, 영어, 중국어', '소속사': 'SM엔터테인먼트', '다른이름': '珉豪; 샤이니; SHINee'}
{'_id': ObjectId('5f17c4855d03ae9996c59551'), '배우이름': '임윤아', '흥행지수': 12577, '출연영화': ['엑시트'], '순위': '12', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}
{'_id': ObjectId('5f17c4855d03ae9996c59550'), '배우이름': '배수지', '흥행지수': 12886, '출연영화': ['백두산', '건축학개론'], '순위': '11', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': '\nhttps://twitter.com/missa_suzy\nhttps://www.instagram.com/skuukzky/\n', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트', '다른이름': '배수지;미쓰에이;'}


### compound 필드 인덱스

In [20]:
actor.create_index([('출연영화', pymongo.TEXT), ('직업', pymongo.TEXT)])

'출연영화_text_직업_text'

In [21]:
docs = actor.find({'$text' : {'$search' : '가수'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c595c3'), '배우이름': '최민호', '흥행지수': 2046, '출연영화': ['장사리 : 잊혀진 영웅들', '인랑'], '순위': '126', '직업': '가수', '생년월일': '1991-12-09', '성별': '남', '신장/체중': '181cm', '학교': '건국대학교 예술학', '취미': '축구, 농구', '특기': '연기, 영어, 중국어', '소속사': 'SM엔터테인먼트', '다른이름': '珉豪; 샤이니; SHINee'}
{'_id': ObjectId('5f17c4855d03ae9996c59551'), '배우이름': '임윤아', '흥행지수': 12577, '출연영화': ['엑시트'], '순위': '12', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}
{'_id': ObjectId('5f17c4855d03ae9996c59550'), '배우이름': '배수지', '흥행지수': 12886, '출연영화': ['백두산', '건축학개론'], '순위': '11', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': '\nhttps://twitter.com/missa_suzy\nhttps://www.instagram.com/skuukzky/\n', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트', '다른이름': '배수지;미쓰에이;'}


In [22]:
docs = actor.find({'$text' : {'$search' : '신과'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('5f17c4855d03ae9996c595ce'), '배우이름': '정지훈', '흥행지수': 1758, '출연영화': ['사자', '덕구', '신과 함께-인과 연'], '순위': '137', '직업': '배우', '생년월일': '2007-05-31', '성별': '남'}
{'_id': ObjectId('5f17c4855d03ae9996c59548'), '배우이름': '마동석', '흥행지수': 19381, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'], '순위': '3', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg', '다른이름': 'Ma Dongseok'}


부분문자열 검색을 위해서는 mongodb index를 사용해야한다!

In [23]:
result = actor.find({'출연영화':{'$regex':'함께'}})
for record in result:
    print(record)

{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('5f17c4855d03ae9996c59548'), '배우이름': '마동석', '흥행지수': 19381, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'], '순위': '3', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg', '다른이름': 'Ma Dongseok'}
{'_id': ObjectId('5f17c4855d03ae9996c595ce'), '배우이름': '정지훈', '흥행지수': 1758, '출연영화': ['사자', '덕구', '신과 함께-인과 연'], '순위': '137', '직업': '배우', '생년월일': '2007-05-31', '성별': '남'}


In [ ]:
# 중앙대학교를 나온 배우 흥행자순으로 정렬

In [50]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_collection'},
 '학교_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine21.actor_collection',
  'weights': SON([('학교', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [46]:
actor.drop_indexes()

In [49]:
actor.create_index([('학교', 'text')])

'학교_text'

In [54]:
docs = actor.find({'$text':{'$search':'중앙대학교'}}).sort('흥행지수', pymongo.DESCENDING)
for doc in docs:
    print(doc) # 중앙대학교라는 부분 문자열을 가진 document만 나옴(ex. 중앙대학교대학원 -> 검색안됨)

{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('5f17c4855d03ae9996c59559'), '배우이름': '박인환', '흥행지수': 10871, '출연영화': ['엑시트', '봄날은 간다', '비밥바룰라', '기묘한 가족', '수상한 그녀'], '순위': '20', '직업': '배우', '생년월일': '1945-01-06', '성별': '남', '학교': '중앙대학교 연극영화과 학사'}
{'_id': ObjectId('5f17c4855d03ae9996c59569'), '배우이름': '박신혜', '흥행지수': 7103, '출연영화': ['＃살아있다', '형', '7번방의 선물', '뷰티 인사이드', '소중한 날의 꿈', '시라노; 연애조작단'], '순위': '36', '직업': '배우', '생년월일': '1990-02-18', '성별': '여', '홈페이지': '\nhttps://twitter.com/ssinz\nhttps://www.instagram.com/ssinz7/\n', '신장/체중': '165cm, 45kg', '학교': '중앙대학교 연극영화학 (재학)', '취미': '음악감상', '특기': '댄스', '소속사': '아바 엔터테인먼트'}
{'_id': ObjectId('5f17c4855d03ae9996c5956f'), '배우이름': '김래원', '흥행지수': 64

In [81]:
docs2 = actor.find({'학교':{'$regex':'중앙대학교'}}).sort('흥행지수', pymongo.DESCENDING)
for doc in docs2:
    print(doc) # 중앙대학교가 들어가기만하면 다추출해줌

{'_id': ObjectId('5f17c4855d03ae9996c59547'), '배우이름': '하정우', '흥행지수': 31716, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '순위': '2', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('5f17c4855d03ae9996c59559'), '배우이름': '박인환', '흥행지수': 10871, '출연영화': ['엑시트', '봄날은 간다', '비밥바룰라', '기묘한 가족', '수상한 그녀'], '순위': '20', '직업': '배우', '생년월일': '1945-01-06', '성별': '남', '학교': '중앙대학교 연극영화과 학사'}
{'_id': ObjectId('5f17c4855d03ae9996c59569'), '배우이름': '박신혜', '흥행지수': 7103, '출연영화': ['＃살아있다', '형', '7번방의 선물', '뷰티 인사이드', '소중한 날의 꿈', '시라노; 연애조작단'], '순위': '36', '직업': '배우', '생년월일': '1990-02-18', '성별': '여', '홈페이지': '\nhttps://twitter.com/ssinz\nhttps://www.instagram.com/ssinz7/\n', '신장/체중': '165cm, 45kg', '학교': '중앙대학교 연극영화학 (재학)', '취미': '음악감상', '특기': '댄스', '소속사': '아바 엔터테인먼트'}
{'_id': ObjectId('5f17c4855d03ae9996c5956f'), '배우이름': '김래원', '흥행지수': 64

In [25]:
import pymongo

conn = pymongo.MongoClient()
actor_db = conn.cine21
text_collection = actor_db.text_collection

In [68]:
text_collection.insert_many(
    [
        { "name": "Java Hut", "description": "Coffee and cakes", "ranking": 1 },
        { "name": "Burger Buns", "description": "Java hamburgers", "ranking": 2 },
        { "name": "Coffee Shop", "description": "Just coffee", "ranking": 3 },
        { "name": "Clothes Clothes Clothes", "description": "Discount clothing", "ranking": 4 },
        { "name": "Java Shopping", "description": "Indonesian goods", "ranking": 5 },
        { "name": "Using Java", "description": "Korean goods", "ranking": 6 }
    ]
)

In [36]:
docs = text_collection.find({'name':{'$regex':'Sh.+'}})
for doc in docs:
    print(doc) # 정규표현식 -> index 지정 필요없음

{'_id': ObjectId('5f1811c57d84c8a50c9756ba'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('5f1811c57d84c8a50c9756bc'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [80]:
docs = text_collection.find({'name':{'$regex':'^C.*'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f18212f7d84c8a50c9756c0'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('5f18212f7d84c8a50c9756c1'), 'name': 'Clothes Clothes Clothes', 'description': 'Discount clothing', 'ranking': 4}


In [59]:
docs = text_collection.find({'name':{'$regex':'Shop'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f1811c57d84c8a50c9756ba'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('5f1811c57d84c8a50c9756bc'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [73]:
docs = text_collection.find({'name':{'$regex':r'^Java'}}) 
for doc in docs:
    print(doc)

{'_id': ObjectId('5f18212f7d84c8a50c9756be'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}
{'_id': ObjectId('5f18212f7d84c8a50c9756c2'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [37]:
text_collection.create_index([('name', pymongo.TEXT)])

'name_text'

In [38]:
text_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.text_collection'},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine21.text_collection',
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [39]:
docs = text_collection.find({'$text':{'$search': 'coffee'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f1811c57d84c8a50c9756ba'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [40]:
docs = text_collection.find({'$text':{'$search': 'java coffe shop'}})
for doc in docs:
    print(doc) # 각각 부분문자열 검색

{'_id': ObjectId('5f1811c57d84c8a50c9756bc'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}
{'_id': ObjectId('5f1811c57d84c8a50c9756b8'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}
{'_id': ObjectId('5f1811c57d84c8a50c9756ba'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [41]:
docs = text_collection.find({'$text':{'$search': '\"coffee shop\"'}})
for doc in docs:
    print(doc) # full sentence 일치하는 document 검색

{'_id': ObjectId('5f1811c57d84c8a50c9756ba'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [42]:
docs = text_collection.find({'$text':{'$search': '\"coffee shop\"', '$caseSensitive':True}})
for doc in docs:
    print(doc) # 소문자 대문자 구분